In [1]:
import re, os
import requests
from urllib.parse import urlparse
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time


In [2]:
root_dir = r'C:\Users\daV\Documents\ZHAW\HS 2024\dPoDL\dPoDL\experiments\filtered-search'

# List all files
all_files = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        all_files.append(file_path)
directories = set([os.path.dirname(i) for i in all_files if not "flagged" in i])
len(all_files)
for file in all_files:
    if file.endswith(".csv"):
        #os.remove(file)
        pass

In [30]:
# Define individual regex patterns for each metric for now just focus on relaxation of problem; LOSS
epoch_loss_pattern = re.compile(r'epoch (\d+)/\d+.*? loss:\s*(\d+\.\d+)', re.DOTALL)
#epoch_loss_val_loss_pattern1 = re.compile(r'epoch (\d+)/\d+.*? loss:\s*(\d+\.\d+).*? val_loss:\s*(\d+\.\d+)' , re.DOTALL)
#epoch_loss_val_loss_pattern2 = re.compile(r'epoch (\d+)/\d+.*? val_loss:\s*(\d+\.\d+).*? loss:\s*(\d+\.\d+)', re.DOTALL)

c = 0

def regex_metrics_tf(text_original):
    text_original = text_original.lower()
    dfs = []
    
    # split everytime new epoch count is detected -> multiple trainings in one notebook
    keyword = "epoch 1/"
    text_splits = re.split(f'(?<={keyword})', text_original)[1:]    # Use regex with lookbehind to keep the keyword in the result
    text_splits = ["epoch 1/" + s  for s in text_splits]
    
    for ix, text in enumerate(text_splits):
        #start = time.time()
        epoch_loss_matches = epoch_loss_pattern.findall(text)
        #print(time.time()-start, "first search")
        
        if len(epoch_loss_matches) > 0:
            df = pd.DataFrame(epoch_loss_matches, columns=["Epoch", "Loss"])
        else:
            df = pd.DataFrame({})
        dfs.append(df)
        
    return dfs


def wrapper(filename):
    file_out = os.path.splitext(filename)[0] + f"-1.csv"
    if os.path.isfile(file_out): 
        return
    file = open(filename, "r").read()        
    a = regex_metrics_tf(file)
    for ix, df in enumerate(a):
        if df.empty:
            print("empty:", filename)
            os.remove(filename)
            break
        else:
            file_out = file_out.replace("-1.csv", f"-{ix+1}.csv")
            df.to_csv(file_out, index=False)
    return a


#test_item = os.listdir()[2322]  #2322   sample 10  -> 82s
#a = wrapper(test_item)

for directory in directories:
    os.chdir(directory)
    for file in [i for i in os.listdir() if ("urls.txt" not in i) and ("flagged" not in i) and (not i.endswith(".csv"))]:
        try:
            wrapper(file)
        except FileNotFoundError:
            print("FileNotFound", file)


FileNotFound Nathan-Austin--Clothing_image_classifier_Tensorflow-Basic_20classification__20Classify_20images_20of_20clothing_20_C2_A0__C2_A0_20TensorFlow_20Core.html


In [11]:
# FILTER OUT POTENTIAL BAD ATTRIBUTION TO "TRANSFORMERS"
import shutil
cnn_keywords = ["cnn", "convolutional"]
potential_bad = []
for filename in all_files:

    if "urls.txt" in filename:
        continue
    if "filtered-search\\transformers" in filename:
        try:
            with open(filename, "r") as f:
                content = f.read().lower()
        except FileNotFoundError:
            continue
            
        new_dir = os.path.join(os.path.dirname(filename), "flagged")
        os.makedirs(new_dir, exist_ok=True)
        if any([True for i in cnn_keywords if i in content]) or any([True for i in cnn_keywords if i in filename]):
            potential_bad.append(filename)
            try:
                shutil.move(filename, os.path.join(new_dir, os.path.basename(filename)))
            except FileNotFoundError:
                pass
